In [ ]:
import sys
sys.path.append('/Users/asabyr/Documents/software/bolocalc-space/analyze-bc')
sys.path.append('../code')
sys.path.append('/Users/asabyr/Documents/software/sd_foregrounds_optimize/')
import numpy as np
from analyze_bands import AnalyzeBands
from analyze_dets import AnalyzeDets
from analyze_calib import AnalyzeCalib
from analyze_sky import AnalyzeSkyModel
from HelpFunctions import *
import copy
from sky_variations_funcs import *

# Fisher forecast settings

In [ ]:
#define some settings for the fisher forecast
snr_settings_mu={}
snr_settings_mu['fraction of the sky observed']=1.
snr_settings_mu['mission integration [months]']=6.
snr_settings_mu['include all foregrounds']=True
snr_settings_mu['set priors']=False
snr_settings_mu['sensitivity file']=None
snr_settings_mu['spectral distortion']='mu'

# Example to analyze band optimization output

In [ ]:
############# try different frequency ranges and initial widths ############# 
############# compute snr for config going down to 25%, 50% and at its peak #############
snr_settings_mu['hemt amps']=True
snr_settings_mu['hemt freq']=10
snr_settings_mu['sensitivity file']=None #to ensure it computes noise directly with bolocalc

mu_1to2000_bolo10_object=AnalyzeBands(filename='../files/mu_1to2000_width1_snr25_bolo10_optimized_bands.txt',
                             snr_settings=snr_settings_mu,
                             min_freq_1=1, max_freq_1=10, dfreq_1=1, freq_step_1=100,
                            min_freq_2=10, max_freq_2=2000, dfreq_2=3, freq_step_2=100)
#reads optimized bands
mu_1to2000_bolo10_025=mu_1to2000_bolo10_object.read_optimized_bands()

#finds bands at specified threshold
mu_1to2000_bolo10_05=mu_1to2000_bolo10_object.read_thresh_bands(thresh=0.5)

#find bands at the peak
mu_1to2000_bolo10_peak=mu_1to2000_bolo10_object.read_peak_bands()

#in this case, the three are all the same; but that is not always the case
#the output dictionaries contain "bands", "freqs", "dets"-- just array of ones
#"noise" (note this is not used anywhere and computes noise for 6 months, fsky=1), "snr_cost" 


In [ ]:
#see what the snr we get using 100 detectors per band, for example
#now we can specify the pre-computed NET sensitivity file to make the computation file
snr_settings_mu['sensitivity file']='../files/mu_1to2000_width1_snr25_bolo10_raw_sensitivity.txt'#this is NET noise
snr_settings_mu['bolocalc file prefix']='none'

snr_cost_max_mu_1to2000=calc_fisher(snr_settings_mu, mu_1to2000_bolo10_025['bands'], 
                                    100.*mu_1to2000_bolo10_025['dets'])
print(snr_cost_max_mu_1to2000[0])

# Example to analyze detector optimization output

In [ ]:
#this step can be slow
#requires 30 "mu_1to2000_width1_snr25_bolo10SNRs_run13_*.txt" files
# and "mu_1to2000_width1_snr25_bolo10_ndet_combinations_run13.txt" 
# (these are too large to include in the repository but could be generated using setup_detector_optimization.py + run_detector_optimization.py
# and the provided ini file)

mu_1to2000_bolo10_run13=AnalyzeDets('mu_1to2000_width1_snr25_bolo10', run=13, nfiles=30)


In [ ]:
#can filter by required snr or focal plane range
mu_1to2000_SNR5_inds_run13=mu_1to2000_bolo10_run13.filter_SNR(req_snr=5.0)
mu_1to2000_inds_run13_cost=mu_1to2000_bolo10_run13.filter_cost(req_cost=[0,0.7])
# mu_1to2000_bolo10_run13.snrs_area[:, 0] contains SNRs
# mu_1to2000_bolo10_run13.snrs_area[:, 1] contains focal plane area
# mu_1to2000_bolo10_run13.dets contains detector combinations 

# Example to analyze calibration output 

In [ ]:
#0.01, 0.001, 0.001 muK_RJ calibration for 16-band set-up
mu_1to2000_bolo10_run30=AnalyzeCalib('mu_1to2000_width1_snr25_bolo10', run=30, nfiles=1)
mu_1to2000_bolo10_run30.maximum_bias()
#contains mu_1to2000_bolo10_run30.bias and mu_1to2000_bolo10_run30.delta_I; bias is listed in both % and sigma
#delta_I is the calibration uncertainty (or any other systematic shift!)

In [ ]:
#0.01, 0.001, 0.001 muK_RJ calibration for 34-band set-up
mu_1to2000_bolo10_run35=AnalyzeCalib('mu_1to2000_width1_snr25_bolo10_34bands', run=35, nfiles=1)
mu_1to2000_bolo10_run35.maximum_bias()


# Example to analyze sky model variation output 

In [ ]:
mu_1to2000_bolo10_run16=AnalyzeSkyModel('mu_1to2000_width1_snr25_bolo10', run=16, nfiles=1, NaNtozero=True)
snr_bias, snr_params=mu_1to2000_bolo10_run16.maximum_snr_drop()
#containts mu_1to2000_bolo10_run16.snr, mu_1to2000_bolo10_run16.params_snr as well as sorted versions 
#can also compute maximum_snr_increase

mu_1to2000_bolo10_run18=AnalyzeSkyModel('mu_1to2000_width1_snr25_bolo10_34bands', run=18, nfiles=1, NaNtozero=True)
snr_bias, snr_params=mu_1to2000_bolo10_run18.maximum_snr_drop()
